### Model

In [33]:
!pip install groq
!pip install langchain_huggingface
!pip install pinecone
!pip install langchain_pinecone

In [34]:
#Groq

from groq import Groq
groq_api_key = os.environ["GROQ_API_KEY"]

client = Groq(api_key = groq_api_key)

### Router

In [35]:
def classify_question(question):
    router_instructions = f"""Você é um especialista em classificar perguntas.
                            Se a pergunta for sobre qualquer coisa que envolva graduação, como normas, diretrizes ou explicações de funcionamento da graduação, classifique como 'vectorstore'.
                            Para outros assuntos classifique como 'calendar_model'.
                            Retorne apenas o rótulo correspondente: 'vectorstore' ou 'calendar_model'.\n\nPergunta: {question}"""

    prompt = (
        router_instructions
    )
    response = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[{"role": "user", "content": prompt}],
        max_completion_tokens=10,
        temperature=1  # Altere o valor da temperatura aqui

    )
    return response.choices[0].message.content

In [21]:
resposta = classify_question("Qual o prazo máximo de conclusão da graduação?")
resposta

'vectorstore'

In [6]:
resposta = classify_question("Qual a data limite para fazer a matrícula?")
resposta

'calendar_model'

### Carregando vecstore

In [36]:
import os
pinecone_api_key = os.environ["PINECONE_API_KEY"]



In [37]:
from langchain_pinecone import PineconeEmbeddings
from pinecone import Pinecone

#importando embeddings
model_name = 'multilingual-e5-large'
embeddings = PineconeEmbeddings(
    model=model_name,
    pinecone_api_key=pinecone_api_key
)

# importando index (banco)

pc = Pinecone(api_key=pinecone_api_key)
index = pc.Index("regulamento")

In [38]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [40]:
def retriever(question):

  retrieved_docs = vector_store.similarity_search("Qual o limite de faltas?")
  return retrieved_docs


In [64]:


def generate_answer_with_rag(question, retrieved_docs):
    # Combine os documentos em um único contexto
    context = "\n".join([doc.page_content for doc in retrieved_docs])

    # Crie o prompt com a pergunta e o contexto
    prompt = (
    "Você é um especialista em responder perguntas com base em documentos fornecidos. "
    "Seu objetivo é gerar uma resposta clara e precisa com base no contexto dado. Responda "
    "utilizando as informações mais relevantes do contexto. Não repita a pergunta e evite frases como 'a resposta é'. "
    "Caso a pergunta não tenha uma resposta direta, forneça informações relacionadas ou direcione para o que for mais relevante, fazendo referência aos artigos utilizados como base\n\n"
    f"Contexto:\n{context}\n\n"
    f"Pergunta: {question}\n\n"
    "Resposta:"
)

    # Envie o prompt para o modelo Groq
    response = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_completion_tokens=300
    )

    # Retorne a resposta gerada
    return response.choices[0].message.content.strip()


In [66]:
pergunta = "Qual o limite de faltas?"

retrieved_docs = retriever(pergunta)

In [67]:
response = generate_answer_with_rag(pergunta, retrieved_docs)
response

'De acordo com o Art. 276, as faltas relativas aos dias em que o estudante estiver em regime de exercícios específicos devem ser registradas no diário de classe, sendo compensadas no momento de consolidação da turma ou na retificação de registros. Além disso, não há menção a um limite de faltas em geral. No entanto, o Art. 120 estabelece que o critério de assiduidade em uma disciplina presencial é satisfeito quando o estudante cumpre a frequência mínima correspondente a 75% da carga horária do componente curricular.'

In [44]:
response

'A pergunta refere-se ao limite de faltas, mas não há um limite explícito definido nos artigos fornecidos. No entanto, podemos inferir que o limite de faltas é o que é necessário para que o estudante não cumpra a frequência mínima correspondente a 75% (setenta e cinco por cento) da carga horária do componente curricular, considerando o rendimento acadêmico exigido (Art. 120). Além disso, as faltas relativas aos dias em que o estudante estiver em regime de exercícios específicos devem ser registradas no diário de classe e compensadas no momento de consolidação da turma ou na retificação de registros (Art. 276, parágrafo único).'